In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time # for sleep time
import csv
import re
import functools
import datetime #for csv file title
from bs4 import BeautifulSoup as bs
import requests

house_title_list = []

driver = webdriver.Chrome(r'/Users/mohamedsellamia/airbnb/chromedriver')

driver.get('https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=filter_change&adults=1&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false')

now = datetime.datetime.now() #for csv file title

#For csv file names
def reduce_concat(x, sep=""):
    return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

def paste(*lists, sep=" ", collapse=None):
    result = map(lambda x: reduce_concat(x, sep=sep), zip(*lists))
    if collapse is not None:
        return reduce_concat(result, sep=collapse)
    return list(result)

#Open CSV Writer
filename = paste(['airBnb_To'], [now.year], ['-'], [now.month], ['-'], [now.day],['.csv'], sep='')
filename = ''.join(filename)
csv_file = open(filename, 'w', encoding='utf-8',newline = '')
writer = csv.writer(csv_file)

index = 1

house_all_links = []

while True:
    
    if index > 17:
        break
    try:
        print("Getting list of links from Page " + str(index))
        index = index + 1
        #driver.maximize_window()
        time.sleep(6) # give new page time to load
                
        house_list = (driver.find_elements_by_xpath('//a[@class="_1wtrbgoo"]'))

        house_links = [house.get_attribute('href') for house in house_list ] 
        
        house_all_links.extend(house_links)
  
        # Locate the next button on the page.
        wait_button = WebDriverWait(driver, 10)
        
        next_button = wait_button.until(EC.element_to_be_clickable((By.XPATH,'//li[@class="_r4n1gzb"]')))
        
        next_button.click()

        
    except Exception as e:
        print(e)
        continue
        
print(house_all_links)
for link in house_all_links: 
    page = requests.get(link)
    soup = bs(page.text, "lxml")
    match = soup.find_all('a', href=True)
    link_list = []
    for i in match:
        link = i["href"]
        link_list.append(link)  
    for url in link_list[2:20]:
        url = ("https://www.airbnb.ca"+str(url))
        airbnb_dict = {}
        driver.get(url)
        time.sleep(7)

        try:
            house_title = driver.find_element_by_xpath('//h1/span[@class="_18hrqvin"]').text
        except:
            house_title = None
        try:
            house_type =  driver.find_element_by_xpath('//div[@class="_504dcb"]').text
        except:
            house_type = None
        try:
            house_location =  driver.find_element_by_xpath('//div[@class="_czm8crp"]').text
        except:
            house_location = None

        try:
            price = driver.find_element_by_xpath('//span/span[@class="_doc79r"]').text
        except:
            price = None        

        try:
            no_of_reviews = driver.find_element_by_xpath('//div[@class="_14hveccc"]').text
        except:
            no_of_reviews = None  

        try:
            rating = driver.find_element_by_xpath('//div[@class="_10za72m2"]').text

        except:
            rating = None

        try:
            max_occupancy = driver.find_elements_by_xpath('//div/div[@class="_36rlri"]/div[@class="_czm8crp"]')
            house_detail = [detail.text for detail in max_occupancy ]

        except:

            house_detail = None
            
        try:
            amenities = driver.find_elements_by_xpath('//*[@id="amenities"]//div[@class ="_czm8crp"]')
            amenity_detail = [amenity.text for amenity in amenities]

        except:

            house_detail = None
        if house_title not in house_title_list:
            house_title_list.append(house_title)
            print ([house_title, house_type, house_location, price, no_of_reviews, rating, house_detail,amenity_detail])
            
            airbnb_dict['house_title'] = house_title
            airbnb_dict['house_type'] = house_type
            airbnb_dict['house_locatione'] = house_location
            airbnb_dict['price'] = price
            airbnb_dict['no_of_reviews'] = no_of_reviews
            airbnb_dict['rating'] = rating
            airbnb_dict['house_detail'] = house_detail
            airbnb_dict['amenity_detail'] = amenity_detail
            writer.writerow(airbnb_dict.values())

csv_file.close()
driver.close()


Getting list of links from Page 1
Getting list of links from Page 2
Getting list of links from Page 3
Getting list of links from Page 4
Getting list of links from Page 5
Getting list of links from Page 6
Getting list of links from Page 7
Getting list of links from Page 8
Getting list of links from Page 9
Getting list of links from Page 10
Getting list of links from Page 11
Getting list of links from Page 12
Getting list of links from Page 13
Getting list of links from Page 14
Getting list of links from Page 15
Getting list of links from Page 16
Getting list of links from Page 17
['https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_type=pagination&place_id=ChIJpTvG15DL1IkRd8S0KlBVNTI&screen_size=large&hide_dates_and_guests_filters=false&adults=1&s_tag=sTnSJbXh', 'https://www.airbnb.com/s/Toronto--ON--Canada/homes?refinement_paths%5B%5D=%2Fhomes&current_tab_id=home_tab&selected_tab_id=home_tab&search_

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=78.0.3904.70)
